In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

print(tf.__version__)

2.6.0


In [5]:
import gdown
scores_train_url_raw="https://drive.google.com/file/d/1Wi_0oRNv5-WmZl20C-B_8CIjM4bsWOaD/view?usp=sharing"
scores_train_url     ='https://drive.google.com/uc?id=' + scores_train_url_raw.split('/')[-2] #Do not change
gdown.download(scores_train_url, 'scores_train.csv',quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1Wi_0oRNv5-WmZl20C-B_8CIjM4bsWOaD
To: /content/scores_train.csv
100%|██████████| 202k/202k [00:00<00:00, 35.5MB/s]


'scores_train.csv'

In [6]:
df_scores = pd.read_csv("scores_train.csv")
print(df_scores.shape)
df_scores.head(20)

(2300, 7)


,Unnamed: 0,User_ID,Activity_ID,No_of_attempts,Time_taken,Translanguage_Level,Usefulness_Score
0,0,0,lab-wendy-shapes-1,1,265.786380,3.093985,0.141203
1,1,0,who-all-videoclip-1,3,261.729561,2.829898,0.084622
2,2,0,farm-poiki-video-2,3,254.504227,2.789455,0.064420
3,3,0,beach-bingo-1,3,258.097252,3.016948,0.008820
4,4,0,farm-poiki-video-1,2,267.432200,2.736853,0.037040
5,5,0,clothes-roulette-1,3,260.184933,2.621569,0.121265
6,6,0,who-poiki-simulation-2,2,269.578557,2.320359,0.380816
7,7,0,colors-bingo-1,4,256.365028,2.522725,0.258336
8,8,0,who-wendy-book-1,1,271.556818,2.544796,0.240149
9,9,0,alphabet-kevin-tracing-1,2,266.726967,2.701406,0.113232


In [7]:
df=df_scores.pivot_table(index="Activity_ID",columns="User_ID",values="Usefulness_Score",aggfunc='mean')
df.head()

User_ID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229
Activity_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
alphabet-kevin-tracing-1,0.113232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.417540,NaN,NaN,0.232467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.024335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.579208,NaN,NaN,0.087115,NaN,NaN,NaN,NaN,NaN,NaN,0.254832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.039721,0.189571,NaN,NaN,0.384969,NaN,NaN,0.592785,NaN,NaN,NaN,NaN,NaN,0.524417,NaN,NaN,NaN,NaN,NaN,NaN
alphabet-poiki-tracing-1,NaN,NaN,NaN,0.179703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.118938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.179703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.415252,0.787012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.784263,NaN,NaN,NaN
beach-bingo-1,0.008820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.672459,NaN,NaN,0.387869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.084629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.208648,NaN,NaN,...,NaN,NaN,0.216742,NaN,NaN,0.181203,NaN,NaN,NaN,NaN,NaN,NaN,0.141203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.586090,NaN,NaN,NaN,NaN,NaN,0.413602,NaN,NaN,NaN,NaN,NaN,0.224812,NaN,NaN,NaN,NaN,NaN,NaN
birthday-wendy-simulation-1,NaN,0.21449,NaN,NaN,NaN,0.230535,NaN,NaN,NaN,0.01322,0.268772,NaN,NaN,0.518696,0.319419,0.219424,0.133046,0.414987,0.784672,NaN,NaN,0.037125,NaN,0.442146,NaN,NaN,0.087994,0.037651,NaN,NaN,NaN,0.435849,NaN,0.065654,0.276191,0.784462,NaN,NaN,NaN,NaN,...,NaN,0.011907,NaN,NaN,NaN,NaN,0.526412,NaN,NaN,NaN,0.526035,NaN,NaN,NaN,NaN,0.181203,0.08712,0.221194,NaN,NaN,NaN,NaN,NaN,0.403108,NaN,NaN,0.068641,NaN,NaN,NaN,0.180201,0.672291,0.233654,NaN,0.409965,NaN,NaN,0.326483,0.269172,0.51556
clothes-roulette-1,0.121265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.435657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
#df['Activity_ID'] = LE.fit_transform(df['Activity_ID'])
df.index=LE.fit_transform(df.index)

In [9]:
#df.head()
df = df.fillna(0)
df.head()

User_ID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229
0,0.113232,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.417540,0.00000,0.000000,0.232467,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024335,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.579208,0.0,0.000000,0.087115,0.000000,0.0,0.0,0.0,0.000000,0.0,0.254832,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.039721,0.189571,0.0,0.000000,0.384969,0.0,0.000000,0.592785,0.0,0.0,0.000000,0.000000,0.000000,0.524417,0.000000,0.0,0.000000,0.000000,0.000000,0.00000
1,0.000000,0.00000,0.0,0.179703,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.118938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.179703,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.415252,0.787012,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.784263,0.000000,0.000000,0.00000
2,0.008820,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.672459,0.00000,0.000000,0.387869,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.084629,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.208648,0.0,0.0,...,0.0,0.000000,0.216742,0.0,0.000000,0.181203,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141203,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.586090,0.0,0.000000,0.000000,0.0,0.000000,0.413602,0.0,0.0,0.000000,0.000000,0.000000,0.224812,0.000000,0.0,0.000000,0.000000,0.000000,0.00000
3,0.000000,0.21449,0.0,0.000000,0.0,0.230535,0.0,0.0,0.000000,0.01322,0.268772,0.000000,0.000000,0.518696,0.319419,0.219424,0.133046,0.414987,0.784672,0.000000,0.0,0.037125,0.0,0.442146,0.0,0.0,0.087994,0.037651,0.0,0.0,0.0,0.435849,0.0,0.065654,0.276191,0.784462,0.0,0.000000,0.0,0.0,...,0.0,0.011907,0.000000,0.0,0.000000,0.000000,0.526412,0.0,0.0,0.0,0.526035,0.0,0.000000,0.000000,0.000000,0.181203,0.08712,0.221194,0.0,0.0,0.000000,0.000000,0.0,0.403108,0.000000,0.0,0.068641,0.000000,0.0,0.0,0.180201,0.672291,0.233654,0.000000,0.409965,0.0,0.000000,0.326483,0.269172,0.51556
4,0.121265,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.435657,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000


In [10]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(df.values)
distances, indices = knn.kneighbors(df.values, n_neighbors=3)

In [11]:
df1 = df.copy()
df1.head()
#df[2][0]

User_ID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229
0,0.113232,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.417540,0.00000,0.000000,0.232467,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024335,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.579208,0.0,0.000000,0.087115,0.000000,0.0,0.0,0.0,0.000000,0.0,0.254832,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.039721,0.189571,0.0,0.000000,0.384969,0.0,0.000000,0.592785,0.0,0.0,0.000000,0.000000,0.000000,0.524417,0.000000,0.0,0.000000,0.000000,0.000000,0.00000
1,0.000000,0.00000,0.0,0.179703,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.118938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.179703,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.415252,0.787012,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.784263,0.000000,0.000000,0.00000
2,0.008820,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.672459,0.00000,0.000000,0.387869,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.084629,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.208648,0.0,0.0,...,0.0,0.000000,0.216742,0.0,0.000000,0.181203,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141203,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.586090,0.0,0.000000,0.000000,0.0,0.000000,0.413602,0.0,0.0,0.000000,0.000000,0.000000,0.224812,0.000000,0.0,0.000000,0.000000,0.000000,0.00000
3,0.000000,0.21449,0.0,0.000000,0.0,0.230535,0.0,0.0,0.000000,0.01322,0.268772,0.000000,0.000000,0.518696,0.319419,0.219424,0.133046,0.414987,0.784672,0.000000,0.0,0.037125,0.0,0.442146,0.0,0.0,0.087994,0.037651,0.0,0.0,0.0,0.435849,0.0,0.065654,0.276191,0.784462,0.0,0.000000,0.0,0.0,...,0.0,0.011907,0.000000,0.0,0.000000,0.000000,0.526412,0.0,0.0,0.0,0.526035,0.0,0.000000,0.000000,0.000000,0.181203,0.08712,0.221194,0.0,0.0,0.000000,0.000000,0.0,0.403108,0.000000,0.0,0.068641,0.000000,0.0,0.0,0.180201,0.672291,0.233654,0.000000,0.409965,0.0,0.000000,0.326483,0.269172,0.51556
4,0.121265,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.435657,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000


In [12]:
df[8][0]
df.index


Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59],
           dtype='int64')

In [13]:
#get recommendations for user 0
user_index = df.columns.tolist().index(0)
number_neighbors=3
# t: movie_title, m: the row number of t in df
#for m in list(enumerate(df.index)):
for m in df.index:
  
  # find movies without ratings by user 0
  if df[user_index][m]==0:
  #if df.iloc[m, user_index] == 0:
    sim_movies = indices[m].tolist()
    movie_distances = distances[m].tolist()
    
    # Generally, this is the case: indices[3] = [3 6 7]. The movie itself is in the first place.
    # In this case, we take off 3 from the list. Then, indices[3] == [6 7] to have the nearest NEIGHBORS in the list. 
    if m in sim_movies:
      id_movie = sim_movies.index(m)
      sim_movies.remove(m)
      movie_distances.pop(id_movie) 

    # However, if the percentage of ratings in the dataset is very low, there are too many 0s in the dataset. 
    # Some movies have all 0 ratings and the movies with all 0s are considered the same movies by NearestNeighbors(). 
    # Then,even the movie itself cannot be included in the indices. 
    # For example, indices[3] = [2 4 7] is possible if movie_2, movie_3, movie_4, and movie_7 have all 0s for their ratings.
    # In that case, we take off the farthest movie in the list. Therefore, 7 is taken off from the list, then indices[3] == [2 4].
    else:
      sim_movies = sim_movies[:n_neighbors-1]
      movie_distances = movie_distances[:n_neighbors-1]
        
    # movie_similarty = 1 - movie_distance    
    movie_similarity = [1-x for x in movie_distances]
    movie_similarity_copy = movie_similarity.copy()
    nominator = 0

    # for each similar movie
    for s in range(0, len(movie_similarity)):
      
      # check if the rating of a similar movie is zero
      if df[user_index][sim_movies[s]] == 0:

        # if the rating is zero, ignore the rating and the similarity in calculating the predicted rating
        if len(movie_similarity_copy) == (number_neighbors - 1):
          movie_similarity_copy.pop(s)
          
        else:
          movie_similarity_copy.pop(s-(len(movie_similarity)-len(movie_similarity_copy)))

      # if the rating is not zero, use the rating and similarity in the calculation
      else:
        nominator = nominator + movie_similarity[s]*df[user_index][sim_movies[s]]

    # check if the number of the ratings with non-zero is positive
    if len(movie_similarity_copy) > 0:
      
      # check if the sum of the ratings of the similar movies is positive.
      if sum(movie_similarity_copy) > 0:
        predicted_r = nominator/sum(movie_similarity_copy)

      # Even if there are some movies for which the ratings are positive, some movies have zero similarity even though they are selected as similar movies.
      # in this case, the predicted rating becomes zero as well  
      else:
        predicted_r = 0

    # if all the ratings of the similar movies are zero, then predicted rating should be zero
    else:
      predicted_r = 0

  # place the predicted rating into the copy of the original dataset
    df1[user_index][m] = predicted_r

In [15]:
for m in df[df[0] > 0][0].index.tolist():
    print(m)

0
2
4
5
22
23
40
50
51
52


In [20]:
def recommend_movies(user, num_recommended_movies):

  print('The list of the Movies {} Has Watched \n'.format(user))

  for m in df[df[user] > 0][user].index.tolist():
    print(m)
  
  print('\n')

  recommended_movies = []

  for m in df[df[user] == 0].index.tolist():

    index_df = df.index.tolist().index(m)
    predicted_rating = df1[df1.columns.tolist().index(user)][index_df]
    recommended_movies.append((m, predicted_rating))

  sorted_rm = sorted(recommended_movies, key=lambda x:x[1], reverse=True)
  
  print('The list of the Recommended Movies \n')
  rank = 1
  for recommended_movie in sorted_rm[:num_recommended_movies]:
    
    print('{}: {} - predicted rating:{}'.format(rank, recommended_movie[0], recommended_movie[1]))
    print('{}: {} - predicted rating:{}'.format(rank, LE.inverse_transform([recommended_movie[0]]), recommended_movie[1]))
    rank = rank + 1

In [21]:
recommend_movies(0, 5)

The list of the Movies 0 Has Watched 

0
2
4
5
22
23
40
50
51
52


The list of the Recommended Movies 

1: 38 - predicted rating:0.3808157598423265
1: ['lab-wendy-book-1'] - predicted rating:0.3808157598423265
2: 37 - predicted rating:0.14120302208520422
2: ['lab-kevin-game-1'] - predicted rating:0.14120302208520422
3: 46 - predicted rating:0.14120302208520422
3: ['shapes-poiki-sequence-1'] - predicted rating:0.14120302208520422
4: 30 - predicted rating:0.061621213903280986
4: ['foodsl-roulette-1'] - predicted rating:0.061621213903280986
5: 6 - predicted rating:0.03703953739386527
5: ['cooking-kevin-simulation-2'] - predicted rating:0.03703953739386527
